# Configuration

## Base Paths

In [7]:
import os
hybridized_code_path = os.getcwd() + "/asuca_sample/hybrid"
reference_code_path = os.getcwd() + "/asuca_sample/reference"

# full ASUCA source directory (cannot be published at this time)
# hybridized_code_path = "/Users/muellermichel/typhoon/Repository/asuca/branches/hybrid/asuca-kij"
# reference_code_path = "/Users/muellermichel/typhoon/Repository/asuca/branches/reference/asuca-kij"

## Code Directories

In [8]:
# Sample directories
hybridized_code_directories = [hybridized_code_path]
reference_code_directories = [reference_code_path]

# Uncomment this for the full ASUCA source
# hybridized_code_directories = [
#     hybridized_code_path + "/HybridSources",
#     hybridized_code_path + "/Framework"
# ]
# reference_code_directories = [
#     reference_code_path + "/Src",
#     reference_code_path + "/Tools"
# ]

# Helper Functions

In [9]:
import re, os

def lines_from_file(path):
    with open(path) as f:
        return f.readlines()
    return None

def areIndexesWithinQuotes(stringToSearch):
    #build up a colored list, showing whether an index inside stringToSearch is in quotes or not.
    #nested quotes such as "hello'world' foobar" are not supported!
    quoteSections = re.split(r'''(['"])''', stringToSearch)
    isStringIndexWithinQuote = isStringIndexWithinQuote = [False] * len(stringToSearch)
    if len(quoteSections) < 2:
        pass
    elif (len(quoteSections) - 1) % 2 != 0:
        raise Exception("Unexpected behavior of regex split. Please check your python version.")
    elif (len(quoteSections) - 1) % 4 != 0: #check re.split documentation to see how it works.
        pass
    else:
        quoteSections.reverse()
        currSection = quoteSections.pop()
        index = len(currSection)
        if index > 0:
            isStringIndexWithinQuote[0:index] = [False] * len(currSection)
        while len(quoteSections) > 0:
            #opening quote part
            currSection = quoteSections.pop()
            sectionLength = len(currSection)
            prefIndex = 0
            if index > 0:
                prefIndex = index
            index = index + sectionLength
            if sectionLength != 1:
                raise Exception("Quote begin marker with strange number of characters")
            isStringIndexWithinQuote[prefIndex:index] = [True]

            #inbetween quotes part
            currSection = quoteSections.pop()
            sectionLength = len(currSection)
            prefIndex = index
            index = index + sectionLength
            isStringIndexWithinQuote[prefIndex:index] = [True] * sectionLength

            #closing quote part
            currSection = quoteSections.pop()
            sectionLength = len(currSection)
            prefIndex = index
            index = index + sectionLength
            if sectionLength != 1:
                raise Exception("Quote end marker with strange number of characters")
            isStringIndexWithinQuote[prefIndex:index] = [True]

            #next part that's not within quotes
            currSection = quoteSections.pop()
            sectionLength = len(currSection)
            prefIndex = index
            index = index + sectionLength
            isStringIndexWithinQuote[prefIndex:index] = [False] * sectionLength
        #sanity check
        if index != len(stringToSearch):
            raise Exception("Index at the end of quotes search is %i. Expected: %i" %(index, len(stringToSearch)))
    return isStringIndexWithinQuote

def findLeftMostOccurrenceNotInsideQuotes(stringToMatch, stringToSearch, leftStartAt=-1, filterOutEmbeddings=False):
    indexesWithinQuotes = areIndexesWithinQuotes(stringToSearch)
    nextLeftStart = leftStartAt + 1
    matchIndex = -1
    for numOfTrys in range(1,101):
        if nextLeftStart >= len(stringToSearch):
            break
        currSlice = stringToSearch[nextLeftStart:]
        matchIndex = currSlice.find(stringToMatch)
        if matchIndex < 0:
            break
        matchEndIndex = matchIndex + len(stringToMatch)
        if not indexesWithinQuotes[nextLeftStart:][matchIndex] \
        and (not filterOutEmbeddings or nextLeftStart + matchIndex < 1 or re.match(r'\W', stringToSearch[nextLeftStart + matchIndex - 1])) \
        and (not filterOutEmbeddings or len(stringToSearch) <= nextLeftStart + matchEndIndex or re.match(r'\W', stringToSearch[nextLeftStart + matchEndIndex])):
            break
        nextLeftStart += matchIndex + 1
        matchIndex = -1
        if numOfTrys >= 100:
            raise Exception("Could not find the string even after 100 tries.")
    return matchIndex + nextLeftStart if matchIndex >= 0 else -1

openMPLinePattern = re.compile(r'\s*\!\$OMP.*', re.IGNORECASE)
openACCLinePattern = re.compile(r'\s*\!\$ACC.*', re.IGNORECASE)
pgiPragmaLinePattern = re.compile(r'\s*\!PGI.*', re.IGNORECASE)

def strip_and_filter_comments(lines):
    filtered = []
    for line in lines:
        if len(line.strip()) == 0:
            continue
        if openMPLinePattern.match(line) or openACCLinePattern.match(line) or pgiPragmaLinePattern.match(line):
            filtered.append(line.strip())
            continue
        commentIndex = findLeftMostOccurrenceNotInsideQuotes("!", line)
        if commentIndex < 0:
            filtered.append(line.strip())
            continue
        if len(line[:commentIndex].strip()) > 0:
            filtered.append(line[:commentIndex].strip())
    return filtered

openMPLinePattern = re.compile(r'\s*\!\$OMP.*', re.IGNORECASE)
openACCLinePattern = re.compile(r'\s*\!\$ACC.*', re.IGNORECASE)
pgiPragmaLinePattern = re.compile(r'\s*\!PGI.*', re.IGNORECASE)
commentedPattern = re.compile(r'^\s*!\s*(.*)', re.IGNORECASE)
emptyLinePattern = re.compile(r'(.*?)(?:[\n\r\f\v]+[ \t]*)+(.*)', re.DOTALL)
multiLineContinuationPattern = re.compile(r'(.*?)\s*\&\s+(?:\!?\$?(?:OMP|ACC)?\&?)?\s*(.*)', re.DOTALL | re.IGNORECASE)

def strip_combine_and_remove_comments(file_obj, keep_comments=False):
    #first pass: strip out commented code (otherwise we could get in trouble when removing line continuations, if there are comments in between)
    remainder = ""
    if not keep_comments:
        noComments = ""
        for line in file_obj:
            if openMPLinePattern.match(line) or openACCLinePattern.match(line) or pgiPragmaLinePattern.match(line):
                noComments += line
                continue
            commentIndex = findLeftMostOccurrenceNotInsideQuotes("!", line)
            if commentIndex < 0:
                noComments += line
                continue
            noComments += line[:commentIndex] + "\n"
        remainder = noComments
    else:
        for line in file_obj:
            commented_match = commentedPattern.match(line)
            if commented_match:
                remainder += ("!%s" %(commented_match.group(1))).strip() + "\n"
                continue
            elif openMPLinePattern.match(line) or openACCLinePattern.match(line) or pgiPragmaLinePattern.match(line):
                remainder += line
                continue
                
            commentIndex = findLeftMostOccurrenceNotInsideQuotes("!", line.strip())
            if commentIndex < 0:
                remainder += line.strip() + "\n"
            else:
                remainder += line.strip()[:commentIndex] + "\n"
    
    #second pass: strip out empty lines (otherwise we could get in trouble when removing line continuations, if there are empty lines in between)
    stripped = "" 
    while True:
        emptyLineMatch = emptyLinePattern.match(remainder)
        if not emptyLineMatch:
            stripped += remainder
            break
        stripped += emptyLineMatch.group(1).strip() + "\n"
        remainder = emptyLineMatch.group(2)

    #third pass: remove line continuations
    remainder = stripped
    output = ""
    while True:
        lineContinuationMatch = multiLineContinuationPattern.match(remainder)
        if not lineContinuationMatch:
            output += remainder
            break
        output += lineContinuationMatch.group(1) + " "
        remainder = lineContinuationMatch.group(2)

    return (output, stripped)

def sanitized_lines_from_file(path, keep_comments=False):
    with open(path) as f:
        combined, stripped_only = strip_combine_and_remove_comments(f, keep_comments)
        return (combined.split("\n")[:-1], stripped_only.split("\n")[:-1])
    return (None, None)

# Source Analysis

## Pattern Matching and Counter Setup

In [10]:
import re
from collections import namedtuple

Analysis = namedtuple(
    "Analysis",
    "stripped, combined, changed, unchanged, subroutine_definitions, calls, imports, data_definitions, save, parameter, openmp, style, parallel_loops, device_data_init, implementation_scheme, commented, radiation, storage_order, data_spec, other"
)

def analyze_source(ref_source_path, hf_source_path):
    ref_lines, ref_with_cont = sanitized_lines_from_file(ref_source_path)
    hf_lines_set = set(sanitized_lines_from_file(hf_source_path, keep_comments=True)[0]) if hf_source_path else set()
    changed_lines = [l for l in ref_lines if l not in hf_lines_set]
    unchanged_lines = [l for l in ref_lines if l in hf_lines_set]
    
    subroutine_definitions = []
    calls = []
    imports = []
    data_definitions = []
    save = []
    parameter = []
    style = []
    openmp = []
    parallel_loops = []
    device_data_init = []
    implementation_scheme = []
    commented = []
    radiation = []
    storage_order = []
    data_spec = []
    other = []
    for c in changed_lines:
        if "storage_order" in ref_source_path:
            storage_order.append(c)
        elif "!%s" %(c) in hf_lines_set:
            commented.append(c)
        elif re.match(r'(^|\W)subroutine\s+', c, re.IGNORECASE):
            subroutine_definitions.append(c)
        elif re.match(r'(^|\W)use\s+', c, re.IGNORECASE):
            imports.append(c)
        elif re.match(r'(.*?\(\/.*|^data)', c, re.IGNORECASE):
            data_definitions.append(c)
        elif re.match(r'.*?\Wsave\W', c, re.IGNORECASE):
            save.append(c)
        elif re.match(r'.*?\parameter\W', c, re.IGNORECASE):
            parameter.append(c)
        elif re.match(r'(^return$|end\s+subroutine\s+\w+|^write\(.*)', c, re.IGNORECASE):
            style.append(c) #JMA often uses superfluous return before subroutine end - we often removed this
            #we also often removed repetition of subroutine name in end statement
            #we also changed the output stream from 6 to 0
        elif re.match(r'^\s*\!\s*\$.*', c, re.IGNORECASE):
            openmp.append(c)
        elif re.match(r'^do\s+[ij].*', c, re.IGNORECASE):
            parallel_loops.append(c)
        elif re.match(r'.*?_hfdev.*|.*?@if.*|.*?@end\s+if.*', c, re.IGNORECASE):
            device_data_init.append(c)
        elif re.match(r'.*?@scheme.*|.*?@end scheme.*', c, re.IGNORECASE):
            implementation_scheme.append(c)
        elif re.match(r'^call\s.*', c, re.IGNORECASE):
            calls.append(c)
        elif "rad_" in ref_source_path and not re.match(r'^\s*@.*|^[\s\w,]*$', c, re.IGNORECASE):
            radiation.append(c)
        elif re.match(r'(^real\W|^(de)?allocate\W)|(:(,)?)+\)\s*\=\s*0\.0', c, re.IGNORECASE):
            data_spec.append(c)
        else:
            other.append(c)
    print("\n".join(other))
    
    return Analysis(
        stripped= len(ref_with_cont),
        combined= len(ref_lines),
        changed= len(changed_lines),
        unchanged= len(unchanged_lines),
        subroutine_definitions= len(subroutine_definitions),
        calls= len(calls),
        imports = len(imports),
        data_definitions= len(data_definitions),
        save= len(save),
        parameter= len(parameter),
        openmp= len(openmp),
        style = len(style),
        parallel_loops= len(parallel_loops),
        device_data_init= len(device_data_init),
        implementation_scheme= len(implementation_scheme),
        commented= len(commented),
        radiation= len(radiation),
        storage_order= len(storage_order),
        data_spec= len(data_spec),
        other= len(other)
    )

## Analysis of Entire Source Directories

In [11]:
def fortran_sources_list(directory_paths):
    import os
    return [
        os.path.join(dp, f)
        for path in directory_paths
        for dp, dn, filenames in os.walk(path)
        for f in filenames if os.path.splitext(f)[1] in ['.f90', '.F90', '.h90', '.H90']
    ]

ignore_list = set([
    "mp_nhm_option_symbol", "pbl_mym_phi",
    "pbl_sib_coupler", "main_gabls3", "main_pp"
])
alias_by_basename = {
    "rad_jma0507":"rad_jma"
}
ref_alias_by_basename = {
    "rad_jma":"rad_jma0507"
}

def analyze_sources(source_directory_paths, comparison_source_directory_paths):
    import os
    comparison_sources_by_basename = {
        os.path.splitext(os.path.basename(path))[0]:path
        for path in fortran_sources_list(comparison_source_directory_paths)
    }
    sources = fortran_sources_list(source_directory_paths)

    # sanitized_lines_from_file(comparison_sources_by_basename["ideal"], keep_comments=True)

    # [l for l in sanitized_lines_from_file(comparison_sources_by_basename["ideal"], keep_comments=True)[0] if "read(10)" in l]

    # print Analysis(*[sum(x) for x in zip(
    #     analyze_source(sources[0], comparison_sources_by_basename["adv"]),
    #     analyze_source(sources[1], comparison_sources_by_basename["coriolis"])
    # )])

    analysis_list = []
    for r in sources:
        basename = os.path.splitext(os.path.basename(r))[0]
        basename = alias_by_basename.get(basename, basename)
        hf_source = comparison_sources_by_basename.get(basename)
        if basename not in ignore_list and not "sib_" in basename:
            print "========== " + basename + " ============="
            print "refpath: %s, hfpath: %s" %(r, hf_source)
            analysis = analyze_source(r, hf_source)
            print analyze_source(r, hf_source)
            analysis_list.append(analysis)
    print "========== TOTAL ============="
    print Analysis(*[sum(x) for x in zip(*analysis_list)])

## Output Generation for Comparison Reference vs. Hybrid
generates "output_reference_vs_hybrid.txt"

In [12]:
analyze_sources(reference_code_directories, hybridized_code_directories)

========== lbc =============
refpath: /Users/muellermichel/Dropbox/Apps/ShareLaTeX/DirectivesWorkshopPaper/data/productivity_analysis/asuca_sample/reference/lbc.f90, hfpath: /Users/muellermichel/Dropbox/Apps/ShareLaTeX/DirectivesWorkshopPaper/data/productivity_analysis/asuca_sample/hybrid/lbc.h90


Analysis(stripped=38, combined=30, changed=7, unchanged=23, subroutine_definitions=1, calls=0, imports=2, data_definitions=0, save=0, parameter=0, openmp=2, style=0, parallel_loops=2, device_data_init=0, implementation_scheme=0, commented=0, radiation=0, storage_order=0, data_spec=0, other=0)
========== surface =============
refpath: /Users/muellermichel/Dropbox/Apps/ShareLaTeX/DirectivesWorkshopPaper/data/productivity_analysis/asuca_sample/reference/surface.f90, hfpath: /Users/muellermichel/Dropbox/Apps/ShareLaTeX/DirectivesWorkshopPaper/data/productivity_analysis/asuca_sample/hybrid/surface.h90


Analysis(stripped=114, combined=86, changed=4, unchanged=82, subroutine_definitions=0, calls=3

## Output Generation for Comparison Hybrid vs. Reference
generates "output_hybrid_vs_reference.txt"

In [13]:
analyze_sources(hybridized_code_directories, reference_code_directories)

========== lbc =============
refpath: /Users/muellermichel/Dropbox/Apps/ShareLaTeX/DirectivesWorkshopPaper/data/productivity_analysis/asuca_sample/hybrid/lbc.h90, hfpath: /Users/muellermichel/Dropbox/Apps/ShareLaTeX/DirectivesWorkshopPaper/data/productivity_analysis/asuca_sample/reference/lbc.f90
@domainDependant{attribute(autoDom, present), accPP(AT_TIGHT_STENCIL), domPP(DOM_TIGHT_STENCIL)}
dens_ref_f, dens_ptb_damp
@end domainDependant
@domainDependant{attribute(autoDom, present), accPP(AT4_TIGHT_STENCIL), domPP(DOM4_TIGHT_STENCIL)}
dens_ptb_bnd
@end domainDependant
@parallelRegion{domName(i,j), domSize(nx_mn:nx_mx,ny_mn:ny_mx), startAt(nx_mn,ny_mn), endAt(nx_mx,ny_mx), template(TIGHT_STENCIL)}
@end parallelRegion
@domainDependant{attribute(autoDom, present), accPP(AT_TIGHT_STENCIL), domPP(DOM_TIGHT_STENCIL)}
dens_ref_f, dens_ptb_damp
@end domainDependant
@domainDependant{attribute(autoDom, present), accPP(AT4_TIGHT_STENCIL), domPP(DOM4_TIGHT_STENCIL)}
dens_ptb_bnd
@end domainDependa

## Finding Routines with Changed Kernel Positions
Requires cpu_routine_positions.json and gpu_routine_positions.json - these were generated using the Hybrid Fortran toolchain applied to the Hybrid ASUCA codebase.

This gives the physics routine names that require a granularity change between CPU and GPU, and thus what needs to be reimplmenented separately in an OpenACC implementation. 

In [14]:
import json

print os.getcwd()

cpu_positions_by_name = {}
with open(os.getcwd() + "/cpu_routine_positions.json") as f:
    cpu_positions_by_name = json.loads(f.read())

gpu_positions_by_name = {}
with open(os.getcwd() + "/gpu_routine_positions.json") as f:
    gpu_positions_by_name = json.loads(f.read())

routines_with_changed_position = []
for routine, position in gpu_positions_by_name.items():
    if routine not in cpu_positions_by_name:
        raise Exception(routine + " not found.")
    if position != cpu_positions_by_name[routine]:
#         print routine, position, cpu_positions_by_name[routine]
        routines_with_changed_position.append(routine)
len(routines_with_changed_position)

/Users/muellermichel/Dropbox/Apps/ShareLaTeX/DirectivesWorkshopPaper/data/productivity_analysis


215

## Finding the LOC for Routines with Changed Kernel Positions

In [15]:
import re

from collections import namedtuple

State = namedtuple("State", "subroutine_name, start_line_no")

def scan_loc_by_routine(source_path):
    lines, _ = sanitized_lines_from_file(source_path)
    state = None
    result = {}
    for line_no, line in enumerate(lines):
        if not state:
            subroutine_match = re.match('^\s*subroutine\s+(\w*).*', line, re.IGNORECASE)
            if subroutine_match:
                state = State(subroutine_match.group(1), line_no)
        else:
            end_subroutine_match = re.match('^\s*end\s*subroutine.*', line, re.IGNORECASE)
            if end_subroutine_match:
                result[state.subroutine_name] = line_no - state.start_line_no
                state = None
    if state:
        raise Exception("unfinished routine %s in %s" %(state.subroutine_name, source_path))
    return result

loc_by_routine = {}
for source in fortran_sources_list(reference_code_directories):
    loc_by_routine.update(scan_loc_by_routine(source))

loc = 0
for routine in routines_with_changed_position:
    alias = ref_alias_by_basename.get(routine, routine)
    if alias in loc_by_routine:
        loc += loc_by_routine[alias]
loc

83